# 第一章 pytorch基础

In [1]:
import torch


In [3]:
a=torch.zeros(2,3)
print(a)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


张量（Tensor）
张量（Tensor）是 PyTorch 中的核心数据结构，用于存储和操作多维数组。

张量可以视为一个多维数组，支持加速计算的操作。

In [4]:
b=torch.randn(2,3)
print(b)

tensor([[ 1.5809,  0.1983, -0.2143],
        [ 2.3237, -0.1553,  0.6574]])


In [8]:
import numpy as np
np_array=np.array([[1,2],[3,4]])
print(np_array)
tensor_from_numpy=torch.from_numpy(np_array)
print(tensor_from_numpy)

[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]], dtype=torch.int32)


In [10]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
d=torch.randn(2,3,device=device)
print(d)

tensor([[-0.7469,  0.1215,  0.5507],
        [ 0.3500, -0.0789, -0.6234]], device='cuda:0')


常用的张量操作

In [11]:
#相加
# 张量相加
e = torch.randn(2, 3)
f = torch.randn(2, 3)
print(e + f)

tensor([[-1.3987,  1.1449,  1.0324],
        [ 0.5543, -1.9386, -1.1582]])


In [18]:
#逐元素乘法，就是对应位置相乘
print("e is ",e)
print("f is ",f)
print(e*f)
print(e.shape)

e is  tensor([[-0.6197,  0.1066,  0.7047],
        [-1.0959, -0.1800, -0.0347]])
f is  tensor([[-0.7790,  1.0384,  0.3276],
        [ 1.6501, -1.7586, -1.1234]])
tensor([[ 0.4827,  0.1107,  0.2309],
        [-1.8083,  0.3165,  0.0390]])
torch.Size([2, 3])


In [17]:
print(e.t())## 张量的转置
print(e.shape)

tensor([[-0.6197, -1.0959],
        [ 0.1066, -0.1800],
        [ 0.7047, -0.0347]])
torch.Size([2, 3])


张量与设备
PyTorch 张量可以存在于不同的设备上，包括CPU和GPU，你可以将张量移动到 GPU 上以加速计算：

if torch.cuda.is_available():
    tensor_gpu = tensor_from_list.to('cuda')  # 将张量移动到GPU

梯度和自动微分


PyTorch的张量支持自动微分，这是深度学习中的关键特性。当你创建一个需要梯度的张量时，PyTorch可以自动计算其梯度：

In [23]:
tensor_requires_grad=torch.tensor([1.0],requires_grad=True)

tensor_result=tensor_requires_grad*3

tensor_result.backward()
print(tensor_requires_grad.grad)

tensor([3.])


torch.Tensor 对象有一个 requires_grad 属性，用于指示是否需要计算该张量的梯度。

当你创建一个 requires_grad=True 的张量时，PyTorch 会自动跟踪所有对它的操作，以便在之后计算梯度。

In [24]:
x=torch.randn(2,2,requires_grad=True)
print(x)

tensor([[ 0.2346, -2.3676],
        [-0.4216, -1.3358]], requires_grad=True)


In [34]:
y=x+2
z=y*y*3
out=z.mean()
print(y)
print(z)
print(out)

tensor([[ 2.2346, -0.3676],
        [ 1.5784,  0.6642]], grad_fn=<AddBackward0>)
tensor([[14.9808,  0.4053],
        [ 7.4738,  1.3234]], grad_fn=<MulBackward0>)
tensor(6.0458, grad_fn=<MeanBackward0>)


In [35]:
out.backward()
print(x.grad)

tensor([[ 6.7039, -1.1027],
        [ 4.7351,  1.9925]])


因为反向传播只能对叶子节点进行啥的，然后y和z是中间变量，然后就不能

In [31]:
z.backward()
print(x.grad)

RuntimeError: grad can be implicitly created only for scalar outputs

停止梯度计算
如果你不希望某些张量的梯度被计算（例如，当你不需要反向传播时），可以使用 torch.no_grad() 或设置 requires_grad=False。

 x * 2

In [36]:
# 使用 torch.no_grad() 禁用梯度计算
with torch.no_grad():
    y = x * 2

In [37]:
import torch.nn as nn
import torch.optim as optim

# 定义一个简单的全连接神经网络
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 2)  # 输入层到隐藏层
        self.fc2 = nn.Linear(2, 1)  # 隐藏层到输出层
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU 激活函数
        x = self.fc2(x)
        return x

# 创建网络实例
model = SimpleNN()

# 打印模型结构
print(model)

SimpleNN(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)


In [45]:
# 随机输入
x=torch.randn(1,2)
print("x ",x)
# 前向传播
output=model(x)
print("output ",output)
# 定义损失函数（例如均方误差 MSE）
criterion=nn.MSELoss()
# 假设目标值为 1
target=torch.randn(1,1)
print("target",target)
# 计算损失
loss=criterion(output,target)
print("loss ",loss)

x  tensor([[ 0.9095, -0.5487]])
output  tensor([[0.5682]], grad_fn=<AddmmBackward0>)
target tensor([[1.0276]])
loss  tensor(0.2111, grad_fn=<MseLossBackward0>)


In [46]:
# 定义优化器（使用 Adam 优化器）
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练步骤
optimizer.zero_grad()  # 清空梯度
loss.backward()  # 反向传播
optimizer.step()  # 更新参数

训练模型
训练模型是机器学习和深度学习中的核心过程，旨在通过大量数据学习模型参数，以便模型能够对新的、未见过的数据做出准确的预测。

训练模型通常包括以下几个步
骤：

数
据准备：

收集和处理数据，包括清洗、标准化和归一化。
将数据分为训练集、

验证集和测试集。
定义模型：

选择模型架构，例如决策树、神经网络等。
初始化

模型参数（权重和偏
置）。
选择损失函数：

根据任务类型（如分

类、回归）选择合
适的损失函数。
选择优化器：

选择一个优化算法，如

SGD、Ada
m等，来更新模型参数。
前向传播：

在每次迭代

中，将输入数据
通过模型传递，计算预测输出。
计算损失：



使用损失函数
评估预测输出与真实标签之间的差异。
反向传

播：

利

用自动求导计算损失相对于模型参数的梯度。
参

数更新：



根据计算出的梯度和优化器的策略更新模型参数。
迭代优化：

重复

步骤5-8，直到

模型在验证集上的性能不再提升或达到预定的迭代次数。


评估和测试：


使用测试集评估模型的最终性能，确保模型没有过拟合。
模

型调优：

根据模型在测试集上的表现进行调参，如改变学习率、增加正则化等。
部署模型：

将训练好的模型部署到生产环境中，用于实际的预测任务。

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 定义一个简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 2)  # 输入层到隐藏层
        self.fc2 = nn.Linear(2, 1)  # 隐藏层到输出层
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU 激活函数
        x = self.fc2(x)
        return x

# 2. 创建模型实例
model = SimpleNN()

# 3. 定义损失函数和优化器
criterion = nn.MSELoss()  # 均方误差损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 优化器

# 4. 假设我们有训练数据 X 和 Y
X = torch.randn(10, 2)  # 10 个样本，2 个特征
Y = torch.randn(10, 1)  # 10 个目标值

# 5. 训练循环
for epoch in range(100):  # 训练 100 轮
    optimizer.zero_grad()  # 清空之前的梯度
    output = model(X)  # 前向传播
    loss = criterion(output, Y)  # 计算损失
    loss.backward()  # 反向传播
    optimizer.step()  # 更新参数
    
    # 每 10 轮输出一次损失
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 1.0289
Epoch [20/100], Loss: 1.0174
Epoch [30/100], Loss: 1.0065
Epoch [40/100], Loss: 0.9961
Epoch [50/100], Loss: 0.9878
Epoch [60/100], Loss: 0.9800
Epoch [70/100], Loss: 0.9727
Epoch [80/100], Loss: 0.9660
Epoch [90/100], Loss: 0.9596
Epoch [100/100], Loss: 0.9536


设备device

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 将模型移动到设备
model.to(device)

# 将数据移动到设备
X = X.to(device)
Y = Y.to(device)